In [20]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
# tf.debugging.set_log_device_placement(False)

logdir = './logs/linreg'
writer = tf.summary.create_file_writer(logdir)

step = 20
slope = 0.4
bias = 1.5
rows = 100

learning_rate = 0.05
steps = 400

x_train = tf.random.uniform(shape=(rows,))
perturb = tf.random.normal(shape=(len(x_train),), stddev=0.01)
y_train = slope * x_train + bias + perturb

m = tf.Variable(0.)
b = tf.Variable(0.)

# @tf.function
def predict_y_value(x):
    y = m * x + b
    return y

# @tf.function
def squared_error(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

# tf.summary.trace_on(graph=True, profiler=True)

for epoch in range(steps):
    with tf.GradientTape() as tape:
        predictions = predict_y_value(x_train)
        loss = squared_error(predictions, y_train)

    gradients = tape.gradient(loss, [m, b])
    m.assign_sub(gradients[0] * learning_rate)
    b.assign_sub(gradients[1] * learning_rate)

    with writer.as_default():
        tf.summary.scalar('training loss', loss, step=epoch+1)
        writer.flush()
    
    # writer.flush()

print("m: %f, b: %f" % (m.numpy(), b.numpy()) )

# with writer.as_default():
#     tf.summary.trace_export(
#         name='linreg',
#         step=0,
#         profiler_outdir=logdir
#     )

TensorFlow version:  2.4.1
m: 0.421551, b: 1.487340


In [21]:
%load_ext tensorboard
%tensorboard --logdir ./logs/linreg

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 85331), started 12:23:15 ago. (Use '!kill 85331' to kill it.)